In [11]:
import torch
import os

from src import *

from tqdm import tqdm
import pickle
import numpy as np

external_path=''

In [12]:
def get_cbv_vectors(positive_digit,negative_digit,num_steps=10000,lr=1e-3):
    if not(os.path.exists(f'{external_path}\\concept_boundary_vectors\\log.txt')):
        log_file=open(f'{external_path}\\concept_boundary_vectors\\log.txt','w')
    else:
        log_file=open(f'{external_path}\\concept_boundary_vectors\\log.txt','a')

    normals=np.load(f'{external_path}\\boundary_info\\normals\\{positive_digit}_{negative_digit}.npy')
    normals=torch.tensor(normals).float()

    optimized_vector=torch.rand((normals.shape[1]))
    optimized_vector=(optimized_vector/torch.norm(optimized_vector)).requires_grad_()

    norm_opt=torch.optim.SGD([optimized_vector],lr=lr)

    steps=0
    while steps<num_steps:
        epoch_loss=0
        for k in range(normals.shape[0]):
            norm_opt.zero_grad()
            dot=torch.dot(optimized_vector,normals[k,:])/torch.norm(optimized_vector)
            loss=torch.abs(1-dot)
            loss.backward()
            norm_opt.step()
            epoch_loss+=loss
            steps+=1
    np.save(f'{external_path}\\concept_boundary_vectors\\{positive_digit}_{negative_digit}.npy',optimized_vector.detach().numpy())
    log_file.write(f'{positive_digit}_{negative_digit}:loss={epoch_loss/normals.shape[0]}')
    log_file.close()

In [16]:
pbar=tqdm(range(10))
for positive_digit in pbar:
    for negative_digit in range(10):
        if positive_digit==negative_digit:
            continue
        pbar.set_description(f'{negative_digit}/9')
        get_cbv_vectors(positive_digit,negative_digit)

8/9: 100%|██████████| 10/10 [06:14<00:00, 37.47s/it]
